In [1]:
import logging
from collections import namedtuple
import random

In [2]:
PROBLEM_SIZE = 5
#First I tried OFFSPRING_SIZE = 30 and NUM_GENERATIONS = 50 for N = 5 but then because the 
# fitness values at the end were wrong, I set OFFSPRING_SIZE to 10 and NUM_GENERATIONS to 5
#for debugging purpose
OFFSPRING_SIZE = 10
NUM_GENERATIONS = 5

Problem instances generator

In [3]:
def problem(N, seed=None):
    """Creates an instance of the problem"""
    state = random.getstate()
    random.seed(seed)
    p = [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]
    random.setstate(state)
    return p

In [4]:
Individual = namedtuple("Individual", ["genome", "fitness"])

#fitness function
def covering(genome):
    numOfElements = 0
    covered = list()
    for G in genome:
        numOfElements += len(G)
        for g in G:
            if(g not in covered):
                covered.append(g)
    return len(covered)/numOfElements

Genetic Algorithm

In [5]:
logging.getLogger().setLevel(logging.INFO)

Initial Population

In [6]:
population = list()
lists = list()
for genome in tuple(problem(PROBLEM_SIZE, 42)):
    population.append(Individual([genome], covering([genome])))
    lists.append(genome)
    
POPULATION_SIZE = len(population)

logging.info(f"init: pop_size={POPULATION_SIZE}; max={max(population, key=lambda i: i.fitness)[1]}")

INFO:root:init: pop_size=25; max=1.0


In [7]:
def tournament(population, tournament_size=2):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)


def cross_over(g1, g2):
    for g in g2:
        if(g not in g1):
            g1.append(g)
            break
    for g in g1:
        if(g not in g2):
            g1.remove(g)
            break
    return g1


def mutation(genome):
    point = random.randint(0, len(lists)-1)
    if(lists[point] in genome):
        if(not len(genome) == 1):
            genome.remove(lists[point])
    else:
        genome.append(lists[point])
    return genome

Evolution

In [8]:
for g in range(NUM_GENERATIONS):
    offspring = list()
    for i in range(OFFSPRING_SIZE):
        if random.random() < 0.3:
            p1 = tournament(population)
            p2 = tournament(population)
            o = cross_over(p1.genome, p2.genome)
        else:
            p = tournament(population)
            o = mutation(p.genome)
        f = covering(o)
        offspring.append(Individual(o, f))
    population += offspring
    population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]